In [6]:
import pandas as pd

def clean_transaction_history():
    df = pd.read_csv('C:\\Users\\sar81\\Downloads\\DownloadTxnHistory (2).csv', skiprows=3)
    df.rename(columns={'TransactionDate': 'trade_date', 'Symbol': 'ticker', 'Quantity': 'quantity', 'Price': 'price'}, inplace=True)
    df.drop(columns=['TransactionType', 'SecurityType', 'Commission', 'Description'], inplace=True)
    df['trade_date'] = pd.to_datetime(df['trade_date'], utc=True, errors='coerce')
    df['trade_type'] = df['quantity'].apply(lambda x: 'buy' if x > 0 else 'sell')
    df['quantity'] = df['quantity'].abs()
    initial_cash_balance = (
        (df.loc[(df['ticker'] == 'SPHY') & (df['trade_date'] == '2025-04-02'), 'price'] * df.loc[(df['ticker'] == 'SPHY') & (df['trade_date'] == '2025-04-02'), 'quantity']).sum() +
        (df.loc[(df['ticker'] == 'SPHY') & (df['trade_date'] == '2025-04-09'), 'price'] * df.loc[(df['ticker'] == 'SPHY') & (df['trade_date'] == '2025-04-09'), 'quantity']).sum() +
        (df.loc[df['quantity'] == 0, 'Amount']).sum() +
        df.loc[df['ticker'] == 'MSBNK', 'quantity'].sum()
    )
    df.drop(columns=['Amount'], inplace=True)
    df = df.loc[df['ticker'] != 'SPHY']
    df = df.loc[df['quantity'] != 0]
    df = df.loc[df['ticker'] != 'MSBNK']
    df = df.sort_values(by=['trade_date','ticker'], ascending=True)
    df.reset_index(drop=True, inplace=True)
    df['id'] = df.index + 1
    df = df[['id', 'ticker', 'trade_type', 'quantity', 'price', 'trade_date']]
    micro = df['trade_date'].dt.strftime('%f').where(df['trade_date'].notna(), '')
    micro5 = micro.str[:5]
    date_time_no_frac = df['trade_date'].dt.strftime('%Y-%m-%d %H:%M:%S').where(df['trade_date'].notna(), '')
    df['trade_date_str'] = ''
    mask = df['trade_date'].notna()
    df.loc[mask, 'trade_date_str'] = date_time_no_frac.loc[mask] + '.' + micro5.loc[mask] + '+00'
    df['trade_date'] = df['trade_date_str']
    df.drop(columns=['trade_date_str'], inplace=True)
    df = df.loc[df['ticker'].isna() == False]
    
    return df, initial_cash_balance

In [7]:
df, initial_cash_balance = clean_transaction_history()

In [8]:
df.to_csv('C:\\Users\\sar81\\Downloads\\cleaned_transaction_history.csv', index=False)